<a href="https://colab.research.google.com/github/slv-ai/Fine-Tune-LLMs-with-DPO/blob/main/SFT_phi2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch datasets accelerate transformers datasets peft bitsandbytes trl einops tqdm #wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
!pip install huggingface_hub -q -U
from huggingface_hub import notebook_login
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 7.5 MB/s eta 0:00:00


In [ ]:
#import wandb

In [ ]:
#wb_token=userdata.get('wb_token')
#wandb.login(key=wb_token)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import pandas as pd
import torch
from datasets import load_dataset,DatasetDict,load_from_disk,Dataset
from peft import LoraConfig,PeftModel,get_peft_model,prepare_model_for_kbit_training,AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer,HfArgumentParser,logging,pipeline
from transformers import DataCollatorForLanguageModeling
from trl import SFTTrainer


Dataset

In [ ]:
dataset=load_dataset("json",data_files="sft_dataset.json",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df=pd.DataFrame(dataset)

In [ ]:
df.head()

,prompt,chosen
0,I feel really alone lately.,"Loneliness can be tough, but there are ways to..."
1,I keep feeling like nobody understands me.,It's important to have people who understand y...
2,"I just want someone to talk to, but I don't kn...",That's a brave step to want to connect with so...
3,Feeling really down and alone.,Loneliness and feeling down can go hand in han...
4,I'm bored and lonely waiting for something to ...,Feeling bored and lonely can be a drag. Have y...


In [ ]:
sft_dataset=dataset.from_pandas(df).train_test_split(test_size=0.1)

In [ ]:
sft_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen'],
        num_rows: 2128
    })
    test: Dataset({
        features: ['prompt', 'chosen'],
        num_rows: 237
    })
})

Formatting Prompt

In [ ]:
def formatting_func(example):
    text = f" user: {example['prompt']}\n # AI therapist: {example['chosen']}"
    return {"text":text}

In [ ]:
train_dataset = sft_dataset["train"].map(formatting_func,remove_columns=["prompt","chosen"])
val_dataset= sft_dataset["test"].map(formatting_func,remove_columns=["prompt","chosen"])


Map:   0%|          | 0/2128 [00:00<?, ? examples/s]

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 2128
})

In [ ]:
#define template for a supervised training
#template="""Below is an conversation between a user and an AI therapist.\n User:{} Response:\n"""
#df['prompt']=df['prompt'].apply(lambda x:template.format(x))
#df['chosen']=df['chosen']+"\n### End"
#df=df[['prompt','chosen']]
#df['text']=df['prompt']+df['chosen']
#df.drop(columns=['prompt','chosen'],inplace=True)


Model

In [ ]:
model_name="microsoft/phi-2"

In [ ]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
#load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23"

)
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


Tokenization

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="left"


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#find appropriate maximum length of input_ids
#import matplotlib.pyplot as plt
#def find_max_length(train_dataset,val_dataset):
#  lengths=[len(x['input_ids']) for x in train_dataset]
#  lengths_val=[len(x['input_ids']) for x in val_dataset]
#  plt.hist(lengths)
#  plt.hist(lengths_val)
#  plt.xlabel('length of input_ids')
#  plt.ylabel('count')
#  plt.legend(['train','val'])
#  plt.show()

#find_max_length(train_dataset,val_dataset)


In [ ]:
# Print out the architecture and named parameters
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, Shape: {param.shape}")
# Print out the textual representation of the model's architecture
print(model)

Parameter name: transformer.embd.wte.weight, Shape: torch.Size([51200, 2560])
Parameter name: transformer.h.0.ln.weight, Shape: torch.Size([2560])
Parameter name: transformer.h.0.ln.bias, Shape: torch.Size([2560])
Parameter name: transformer.h.0.mixer.Wqkv.weight, Shape: torch.Size([9830400, 1])
Parameter name: transformer.h.0.mixer.Wqkv.bias, Shape: torch.Size([7680])
Parameter name: transformer.h.0.mixer.out_proj.weight, Shape: torch.Size([3276800, 1])
Parameter name: transformer.h.0.mixer.out_proj.bias, Shape: torch.Size([2560])
Parameter name: transformer.h.0.mlp.fc1.weight, Shape: torch.Size([13107200, 1])
Parameter name: transformer.h.0.mlp.fc1.bias, Shape: torch.Size([10240])
Parameter name: transformer.h.0.mlp.fc2.weight, Shape: torch.Size([13107200, 1])
Parameter name: transformer.h.0.mlp.fc2.bias, Shape: torch.Size([2560])
Parameter name: transformer.h.1.ln.weight, Shape: torch.Size([2560])
Parameter name: transformer.h.1.ln.bias, Shape: torch.Size([2560])
Parameter name: tra

LoRA

In [ ]:

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules= ["Wqkv", "fc1", "fc2" ],
    task_type="CAUSAL_LM",
)


Supervised Fine Tuning

In [ ]:
model=get_peft_model(model,peft_config)
model.print_trainable_parameters()


trainable params: 73,400,320 || all params: 2,853,084,160 || trainable%: 2.5726657849448085


In [ ]:
training_args=TrainingArguments(
    output_dir="phi-2",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    max_grad_norm=0.3,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    learning_rate=2e-4,
    save_steps=100,
    max_steps=1000,
    lr_scheduler_type="cosine",
    warmup_ratio=0.001,
    logging_steps=10
    )

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=500,
    tokenizer=tokenizer,
    args=training_args,

)

Map:   0%|          | 0/2128 [00:00<?, ? examples/s]

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
10,2.115400
20,1.719100
30,1.713300
40,1.620600
50,1.302700
60,1.569500
70,1.592600
80,1.404700
90,1.360900
100,1.670500


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not s

TrainOutput(global_step=1000, training_loss=1.3481568670272828, metrics={'train_runtime': 369.1466, 'train_samples_per_second': 2.709, 'train_steps_per_second': 2.709, 'total_flos': 2042962674493440.0, 'train_loss': 1.3481568670272828, 'epoch': 0.47})

In [ ]:
#trainer.evaluate()

{'eval_loss': 1.4107145071029663,
 'eval_runtime': 57.9248,
 'eval_samples_per_second': 8.166,
 'eval_steps_per_second': 8.166,
 'epoch': 0.05}

save model


In [ ]:
#trainer.save_model("phi2-finetuned")

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.


In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir results/runs

Reload model and merge

In [ ]:
#clear the memory
del model,trainer
torch.cuda.empty_cache()

In [ ]:
ft_model=AutoPeftModelForCausalLM.from_pretrained(
    "phi-2/checkpoint-900",
    device_map="auto",
    torch_dtype=torch.bfloat16
)
ft_model=ft_model.merge_and_unload()


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
ft_model.save_pretrained("phi-2-sft-finetuned")

In [ ]:
model_path='Porselvi/phi-2-finetuned'
ft_model.push_to_hub(model_path,token=True,safe_serialization=True)
tokenizer.push_to_hub(model_path,token=True,safe_serialization=True)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Porselvi/phi-2-finetuned/commit/e3a574820b670ef9e8b0f87d294183315a0f51ce', commit_message='Upload tokenizer', commit_description='', oid='e3a574820b670ef9e8b0f87d294183315a0f51ce', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
torch.cuda.empty_cache()